## Imports

In [44]:
import firebase_admin
from firebase_admin import firestore
from firebase_admin import credentials
from google.cloud.firestore import GeoPoint
from google.cloud.firestore_v1.base_query import FieldFilter

import pytz
import numpy as np
from pprint import pprint 
from itertools import combinations
from geopy.distance import geodesic
from scipy.spatial import ConvexHull
from datetime import datetime, timedelta


## Params

In [47]:
thres_distance = 1000
creds_path = "./keys/serviceAccountKey.json"
indian_timezone = pytz.timezone('Asia/Kolkata')
DEBUG = True

## utils

In [ ]:
def calculate_convex_hull_area(coords):
    hull = ConvexHull(coords)
    return hull.volume  # For 2D convex hull, use hull.area

In [54]:
def select_bounding_coords(coordinates:list)->list:
    max_area = 0
    max_area_coords = None

    # Iterate through all combinations of coordinates
    for i in range(len(coordinates), -1,-1):
        for comb in combinations(coordinates, i):  # Assuming you want to form a triangle (change to 4 for quadrilateral, etc.)
            if not all(c in comb for c in coordinates):  # Skip combinations where not all coordinates are included
                continue
            area = calculate_convex_hull_area(np.array(comb))
            if area > max_area:
                max_area = area
                max_area_coords = comb

    coordList = [GeoPoint(x[0], x[1]) for x in max_area_coords]
    
    return coordList

In [55]:
def get_dict_by_regionMapId(regionMap_id):
    for region_map in regionMaps:
        if region_map['regionMap_id'] == regionMap_id:
            return region_map
    return None

In [56]:
def get_marker_cord_by_id(marker_id):
    for marker in markers_:
        if marker['marker-id'] == marker_id:
            return GeoPoint(marker['marker_cord'][0],marker['marker_cord'][1])
    return None

In [57]:
def get_marker_cord_by_id_tuple(marker_id):
    for marker in markers_:
        if marker['marker-id'] == marker_id:
            return marker['marker_cord']
    return None

In [46]:
# if DEBUG:
#     import matplotlib.pyplot as plt
#     x, y = zip(*coordinates)
#     plt.scatter(x, y, color='blue', label='Original Coordinates')
    
#     # Plot the convex hull with maximum area
#     hull = ConvexHull(np.array(max_area_coords))
#     for simplex in hull.simplices:
#         plt.plot(hull.points[simplex, 0], hull.points[simplex, 1], 'k-')
    
#     plt.scatter(hull.points[hull.vertices, 0], hull.points[hull.vertices, 1], color='red', label='Convex Hull Vertices')
    
#     plt.title('Convex Hull with Maximum Area')
#     plt.xlabel('Latitude')
#     plt.ylabel('Longitude')
#     plt.legend()
#     plt.show()

## Connect

In [5]:
cred = credentials.Certificate(creds_path)
firebase_admin.initialize_app(cred)
db = firestore.client()

## Markers

In [48]:
current_datetime = datetime.now()
midnight_datetime = current_datetime.replace(hour=0, minute=0, second=0, microsecond=0)

one_day = timedelta(days=1)
yesterday_midnight_datetime = current_datetime - one_day
yesterday_midnight_datetime = yesterday_midnight_datetime.replace(hour=0, minute=0, second=0, microsecond=0)

start_date = indian_timezone.localize(yesterday_midnight_datetime)
end_date = indian_timezone.localize(midnight_datetime)


In [51]:
markers_ = []
markers_get = db.collection('Markers').where(filter=FieldFilter('time', ">=", start_date)).where(filter=FieldFilter('time', "<", end_date)).get()
# markers_get = db.collection('Markers').get()
for m in markers_get:
    marker = {
        "marker-id":m.to_dict()["id"],
        "marker_cord":(m.to_dict()["lat"],m.to_dict()["long"]),
        }
    markers_.append(marker)

In [52]:
if DEBUG:
    pprint(markers_)

[{'marker-id': 1702827474781,
  'marker_cord': (26.143074218982125, 91.76195416599512)},
 {'marker-id': 1702831179224, 'marker_cord': (26.5276161, 93.885418)},
 {'marker-id': 1702831353978,
  'marker_cord': (26.523327432461524, 93.88401668518782)},
 {'marker-id': 1702831370550,
  'marker_cord': (26.525289944991446, 93.88164695352316)}]


## Region Maps

In [53]:
regionMaps = []

In [10]:
regionMaps_get = db.collection('RegionMap').get()
for m in regionMaps_get:
    data = m.to_dict()
    data["regionMap_id"]= m.id
    regionMaps.append(data)

In [11]:
if DEBUG:
    pprint(regionMaps)

[{'central_coord': <google.cloud.firestore_v1._helpers.GeoPoint object at 0x7fa6c652fc70>,
  'coords': [<google.cloud.firestore_v1._helpers.GeoPoint object at 0x7fa6c652f910>,
  'markers': [1702827474781, 1702772289395],
  'regionMap_id': 'NzlIV6KigFRGTrCgZ8Gx'},
 {'central_coord': <google.cloud.firestore_v1._helpers.GeoPoint object at 0x7fa6b13c7340>,
  'coords': [<google.cloud.firestore_v1._helpers.GeoPoint object at 0x7fa6b13c5ba0>],
  'markers': [1702840841383],
  'regionMap_id': 'Ud5VSkLhIDnXDSsxFydX'},
 {'central_coord': <google.cloud.firestore_v1._helpers.GeoPoint object at 0x7fa6b13c5e40>,
  'coords': [<google.cloud.firestore_v1._helpers.GeoPoint object at 0x7fa6b13c5fc0>,
  'markers': [1702831353978, 1702831179224, 1702831370550, 1702837908940],
  'regionMap_id': 'qV2bxi5cFie31VszE8Zk'}]


In [12]:
for r in regionMaps:
    r['central_coord'] = (r['central_coord'].to_protobuf().latitude, r['central_coord'].to_protobuf().longitude)
    r['coords'] = [(x.to_protobuf().latitude, x.to_protobuf().longitude)for x in r['coords']]

In [13]:
if DEBUG:
    pprint(regionMaps)

[{'central_coord': (26.14139595138026, 91.76107054576278),
  'coords': [(26.143074218982125, 91.76195416599512),
             (26.139717683778397, 91.76018692553043)],
  'markers': [1702827474781, 1702772289395],
  'regionMap_id': 'NzlIV6KigFRGTrCgZ8Gx'},
 {'central_coord': (26.300955599413115, 93.96943598985672),
  'coords': [(26.300955599413115, 93.96943598985672)],
  'markers': [1702840841383],
  'regionMap_id': 'Ud5VSkLhIDnXDSsxFydX'},
 {'central_coord': (26.526254545900574, 93.88349554741812),
  'coords': [(26.523327432461524, 93.88401668518782),
             (26.5276161, 93.885418),
             (26.525289944991446, 93.88164695352316),
             (26.528784706149324, 93.8829005509615)],
  'markers': [1702831353978, 1702831179224, 1702831370550, 1702837908940],
  'regionMap_id': 'qV2bxi5cFie31VszE8Zk'}]


## check for which region map is the marker

In [34]:
def is_within_distance(coord1, coord2, max_distance=thres_distance):
    # Calculate distance in meters using geopy
    distance = geodesic(coord1, coord2).meters
    return distance <= max_distance

In [35]:
parings = {}

In [36]:
for marker in markers_:
    marker_coord = marker['marker_cord']
    for region_map in regionMaps:
        for coord in region_map['coords']:
            if is_within_distance(marker_coord, coord):
                parings[marker['marker-id']]=region_map['regionMap_id']
                

In [37]:
if DEBUG:
    pprint(parings)

{1702772289395: 'NzlIV6KigFRGTrCgZ8Gx',
 1702827474781: 'NzlIV6KigFRGTrCgZ8Gx',
 1702831179224: 'qV2bxi5cFie31VszE8Zk',
 1702831353978: 'qV2bxi5cFie31VszE8Zk',
 1702831370550: 'qV2bxi5cFie31VszE8Zk',
 1702837908940: 'qV2bxi5cFie31VszE8Zk',
 1702840841383: 'Ud5VSkLhIDnXDSsxFydX'}


In [33]:
for key in parings.keys():
    result_dict = get_dict_by_regionMapId(parings[key])
    ref = db.collection("RegionMap").document(parings[key])
    if DEBUG:
        print(ref.get().to_dict())
    result_dict["coords"].append(get_marker_cord_by_id_tuple(key))
    
    try:
        list_bbox = select_bounding_coords(result_dict["coords"])

        not_list_box = [GeoPoint(x[0],x[1])  for x in result_dict["coords"] if GeoPoint(x[0],x[1]) not in list_bbox]
    except Exception as e:
        if DEBUG:
            print(e)
        not_list_box= []
        
    if DEBUG:
        pprint(not_list_box)
    
    average_latitude = sum(lat for lat, lon in result_dict["coords"]) / len(result_dict["coords"])
    average_longitude = sum(lon for lat, lon in result_dict["coords"]) / len(result_dict["coords"])

    central_coord = GeoPoint(average_latitude, average_longitude)
    ref.update({
        "central_coord": central_coord,
        "coords": firestore.ArrayUnion([get_marker_cord_by_id(key)]),
        "markers": firestore.ArrayUnion([key])
        })
    if len(not_list_box)>0:
        ref.update({
            "coords": firestore.ArrayRemove(not_list_box),
            })

{'central_coord': <google.cloud.firestore_v1._helpers.GeoPoint object at 0x7fa64bfc21a0>, 'markers': [1702831353978, 1702831179224, 1702831370550, 1702837908940], 'coords': [<google.cloud.firestore_v1._helpers.GeoPoint object at 0x7fa64bfc2cb0>, <google.cloud.firestore_v1._helpers.GeoPoint object at 0x7fa64bfc10c0>, <google.cloud.firestore_v1._helpers.GeoPoint object at 0x7fa64bfc26b0>, <google.cloud.firestore_v1._helpers.GeoPoint object at 0x7fa64bfc1150>]}
[]
{'central_coord': <google.cloud.firestore_v1._helpers.GeoPoint object at 0x7fa64bfc1780>, 'markers': [1702827474781, 1702772289395], 'coords': [<google.cloud.firestore_v1._helpers.GeoPoint object at 0x7fa64bfc3220>, <google.cloud.firestore_v1._helpers.GeoPoint object at 0x7fa64bfc1600>]}
QH6154 Qhull precision error: Initial simplex is flat (facet 1 is coplanar with the interior point)

While executing:  | qhull i Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 169350903  incidence  Qtriangulate  _pre-merge  _zero-ce

## remove updated markers

In [38]:
filtered_markers = [marker for marker in markers_ if marker['marker-id'] not in parings.keys()]

In [39]:
if DEBUG:
    pprint(filtered_markers)

[]


## create region map

In [40]:
coordinates = [entry['marker_cord'] for entry in filtered_markers]

# Create a list to store the clusters of coordinates
coordinate_clusters = []

# Iterate through each coordinate
for coord in coordinates:
    # Check if the coordinate is within 500 meters of any existing cluster
    found_cluster = False
    for cluster in coordinate_clusters:
        if any(is_within_distance(coord, c) for c in cluster):
            cluster.append(coord)
            found_cluster = True
            break

    # If not within 500 meters of any existing cluster, create a new cluster
    if not found_cluster:
        coordinate_clusters.append([coord])

## Add new region maps

In [41]:
region_data = []
for coords_lst in coordinate_clusters:
    marker_ids = [entry['marker-id'] for entry in markers_ if entry['marker_cord'] in coords_lst]
    
    average_latitude = sum(lat for lat, lon in coords_lst) / len(coords_lst)
    average_longitude = sum(lon for lat, lon in coords_lst) / len(coords_lst)

    central_coord = GeoPoint(average_latitude, average_longitude)

    data = {
        "central_coord": central_coord,
        "coords": [GeoPoint(x[0],x[1]) for x in coords_lst],
        "markers":marker_ids
    }
    region_data.append(data)

In [42]:
# create new region_maps
for data in  region_data:
    update_time, rm_ref = db.collection('RegionMap').add(data)
    if DEBUG:
        print(f"Added document with id {rm_ref.id}")